# PyOTI Phishing Triage
***
## Installation

```bash
python3 -m pip install virtualenv
git clone https://github.com/RH-ISAC/PyOTI
cd PyOTI
python3 -m venv venv
source venv/bin/activate
python3 -m pip install -r requirements.txt
python3 -m pip install .
```
>If you experience issues installing pycares, please uninstall c-ares from ```/usr/local``` or run ```brew uninstall --ignore-dependencies c-ares```. Pycares depends on the bundled version. (https://github.com/ccxt/ccxt/issues/4798)

## API Keys
Set your API key variables below:

In [ ]:
domaintools = '{USER}:{SECRET}'
googlesafebrowsing = ''
hybridanalysis = ''
phishtank = ''
urlscan = ''
virustotal = ''

***
## URL Analysis
Set the suspicious URL variable below:

>If the URL contains base64-encoded username/address please replace with ``` redacted@redacted.com ``` or ``` cmVkYWN0ZWRAcmVkYWN0ZWQuY29tCg== ```

**Suspicious URL**

In [ ]:
# we import this to safely display the suspicious URL to avoid accidental clicks
from defang import defang

phish_url = ''

**Iris Investigate**

In [ ]:
from pyoti.domains import IrisInvestigate
from pyoti.utils import split_url_domain

iris = IrisInvestigate(api_key=domaintools)
phish_domain = split_url_domain(phish_url)
iris.domain = phish_domain
domain = iris.check_domain()

print(f"[+] Domain risk score for {phish_domain}: {domain[0]['domain_risk']['risk_score']}")


**Google Safe Browsing**

In [ ]:
from pyoti.urls import GoogleSafeBrowsing

gsb = GoogleSafeBrowsing(api_key=googlesafebrowsing)
gsb.url = phish_url
url = gsb.check_url()

if url['matches']:
    print(f"[+] Threat Type: {url['matches'][0]['threatType']}")
    print(f"[+] Platform Type: {url['matches'][0]['platformType']}")
else:
    print(f"[*] No results for {defang(phish_url)}!")

**Hybrid Analysis**

In [ ]:
from pyoti.multis import HybridAnalysis

ha = HybridAnalysis(api_key=hybridanalysis)
ha.url = phish_url
url = ha.check_url()

if url:
    print(f"[+] Hybrid Analysis verdict: {url[0]['verdict']}")
    print(f"[+] Date of analysis: {url[0]['analysis_start_time']}")
    print(f"[*] Link to analysis: https://www.hybrid-analysis.com/sample/{url[0]['sha256']}")
else:
    print(f"[*] No results for {defang(phish_url)}!")

**Phishtank**

In [ ]:
from pyoti.urls import Phishtank

pt = Phishtank(api_key=phishtank)
pt.url = phish_url
url = pt.check_url()

if url['in_database'] == 'true':
    print(f"[+] Valid: {url['valid']}")
    print(f"[+] Verified: {url['verified']}")
    print(f"[+] Date Verified: {url['verified_at']}")
    print(f"[*] Link to analysis: {url['phish_detail_page']}")
else:
    print(f"[*] No results for {defang(phish_url)}!")

**URLhaus**

In [ ]:
import pandas

from pyoti.multis import URLhaus

urlhaus = URLhaus()
urlhaus.url = phish_url
url = urlhaus.check_url()

if url['query_status'] == 'ok':
    print(f"[+] URL threat: {url['threat']}")
    print(f"[+] URL status: {url['url_status']}")
    print(f"[+] Date added: {url['date_added']}")
    print(f"[+] Tags: {[i for i in url['tags']]}")
    print("[*] Payload delivery:")
    data = url['payloads']
    df = pandas.DataFrame.from_dict(data)
    df_payloads = df[['firstseen', 'file_type', 'response_sha256', 'signature']]
    print(df_payloads.to_string())
    print(f"[*] Link to analysis: {url['urlhaus_reference']}")
else:
   print(f"[*] No results for {defang(phish_url)}!")

**URLscan**

Search URLscan to see if URL has been submitted already

In [ ]:
from pyoti.multis import URLscan

urls = URLscan(api_key=urlscan)
urls.url = phish_url
tasks = urls.search_url()

count = 1
for i in tasks['results']:
    print(f"Result #{count}:")
    print(f"\tIndexed at: {i['indexedAt']}")
    print(f"\tTask UUID: {i['task']['uuid']}")
    count += 1

Copy the Task UUID and set the task_uuid variable to check URLscan for results

In [ ]:
task_uuid = 'bba8eac4-209a-40e2-b270-b3d8cc2d3e64'

task_url = urls.check_url(uuid=task_uuid)
print(f"[+] URLscan Verdict score: {task_url['verdicts']['urlscan']['score']}")
if task_url['verdicts']['urlscan']['categories']:
    print(f"[+] Categories: {task_url['verdicts']['urlscan']['categories']}")
if task_url['verdicts']['urlscan']['brands']:
    print(f"[+] Brands: {task_url['verdicts']['urlscan']['brands']}")
if task_url['verdicts']['urlscan']['tags']:
    print(f"[+] Tags: {task_url['verdicts']['urlscan']['tags']}")
print(f"[+] Malicious: {task_url['verdicts']['urlscan']['malicious']}")
print(f"[+] ASN: {task_url['page']['asn']}")
print(f"[+] ASN Name: {task_url['page']['asnname']}")
print(f"[+] Country: {task_url['page']['country']}")
print(f"[+] Server: {task_url['page']['server']}")
print(f"[+] IP: {task_url['page']['ip']}")
print(f"[*] Link to analysis: {task_url['task']['reportURL']}")

**VirusTotal**

In [ ]:
from pyoti.multis import VirusTotal

vt = VirusTotal(api_key=virustotal)
vt.url = phish_url
url = vt.check_url()

if url['response_code'] == 1:
    print(f"[+] Scan date: {url['scan_date']}")
    print(f"[+] Positives: {url['positives']}")
    print(f"[+] Total: {url['total']}")
    print(f"[*] Link to analysis: {url['permalink']}")
else:
    print(f"[*] Verbose Message: {url['verbose_msg']}")